<a href="https://colab.research.google.com/github/gzc/spark/blob/main/project_2/project_2_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2

In this project, we are going to play with Twitter data.

The data is represented as rows of of [JSON](https://en.wikipedia.org/wiki/JSON#Example) strings.
It consists of [tweets](https://dev.twitter.com/overview/api/tweets), [messages](https://dev.twitter.com/streaming/overview/messages-types), and a small amount of broken data (cannot be parsed as JSON).

For this project, we will only focus on tweets and ignore all other messages. I recommend using [ujson](https://pypi.python.org/pypi/ujson) instead of json. It is at least 15x faster than the built-in json library according to our tests.

## Important Reminders

1. The tokenizer in this notebook contains UTF-8 characters. So the first line of your `.py` source code must be `# -*- coding: utf-8 -*-` to define its encoding. Learn more about this topic [here](https://www.python.org/dev/peps/pep-0263/).
2. The input files (the tweets) contain UTF-8 characters. So you have to correctly encode your input with some function like `lambda text: text.encode('utf-8')`.

## Tweets

A tweet consists of many data fields. You can learn all about them in the Twitter API doc. We are going to briefly introduce here.

* `created_at`: Posted time of this tweet (time zone is included)
* `id_str`: Tweet ID - we recommend using `id_str` over using `id` as Tweet IDs, becauase `id` is an integer and may bring some overflow problems.
* `text`: Tweet content
* `user`: A JSON object for information about the author of the tweet
* `id`: User ID
* `name`: User name (may contain spaces)
* `screen_name`: User screen name (no spaces)
* `entities`: A JSON object for all entities in this tweet
* `hashtags`: An array for all the hashtags that are mentioned in this tweet
* `urls`: An array for all the URLs that are mentioned in this tweet


```
{
  "created_at": "Thu Apr 06 15:24:15 +0000 2017",
  "id_str": "850006245121695744",
  "text": "1\/ Today we\u2019re sharing our vision for the future of the Twitter API platform!\nhttps:\/\/t.co\/XweGngmxlP",
  "user": {
    "id": 2244994945,
    "name": "Twitter Dev",
    "screen_name": "TwitterDev",
    "location": "Internet",
    "url": "https:\/\/dev.twitter.com\/",
    "description": "Your official source for Twitter Platform news, updates & events. Need technical help? Visit https:\/\/twittercommunity.com\/ \u2328\ufe0f #TapIntoTwitter"
  },
  "place": {   
  },
  "entities": {
    "hashtags": [      
    ],
    "urls": [
      {
        "url": "https:\/\/t.co\/XweGngmxlP",
        "unwound": {
          "url": "https:\/\/cards.twitter.com\/cards\/18ce53wgo4h\/3xo1c",
          "title": "Building the Future of the Twitter API Platform"
        }
      }
    ],
    "user_mentions": [     
    ]
  }
}
```

# Part 0: Load data to a RDD

1. Make RDD from the input file `tweets.txt`.
2. call the `print_count` method to print number of lines in all these files

It should print
```
Number of elements: 4
```

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz  
!tar xf /content/spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
# -*- coding: utf-8 -*-
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
# coding: utf-8

In [ ]:
def print_count(rdd):
    print ('Number of elements:', rdd.count())

Download the dataset from [here](https://github.com/gzc/spark/blob/main/project_2/tweets.txt) and keep it somewhere on your computer. Load the dataset into your Colab directory from your local system:

In [ ]:
from google.colab import files
files.upload()

Saving tweets.txt to tweets (5).txt


{'tweets.txt': b'{"created_at": "Wed Oct 10 20:19:24 +0000 2018","id_str": "1050118621198921728", "text": "To make room for more expression, we will now count all emojis as equal\xe2\x80\x94including those with gender and skin t\xe2\x80\xa6 https://t.co/MkGjXf9aXm", "user": {"id": 1314}, "entities": {}}\n{"created_at": "Wed Oct 10 20:19:24 +0000 2019", "id_str": "1050118621198921729", "text": "Hello world", "user": {"id": 1314}, "entities": {}}\n{"created_at": "Wed Oct 10 20:19:24 +0000 2019", "id_str": "1050118621198921730", "text": "fly fly everyday", "user": {"id": 20000}, "entities": {}}\n{"created_at": "Thu Apr 06 15:24:15 +0000 2017","id_str": "850006245121695744","text": "1\\/ Today we\\u2019re sharing our vision for the future of the Twitter API platform!\\nhttps:\\/\\/t.co\\/XweGngmxlP","user": {"id": 2244994945,"name": "Twitter Dev","screen_name": "TwitterDev","location": "Internet","url": "https:\\/\\/dev.twitter.com\\/","description": "Your official source for Twitter Platf

In [ ]:
files = ['tweets.txt']
contents = sc.textFile(','.join(files)).map(lambda x:x.encode('utf-8'))
print_count(contents)

Number of elements: 5


# Part 1: Parse JSON strings to JSON objects

Python has built-in support for JSON.
Python built-in json library is too slow. Therefore we recommend using [ujson](https://pypi.python.org/pypi/ujson) instead of json.


In [ ]:
!pip install ujson
import ujson
print("-------------------------------------------------")

json_example = '''
    {
    "id": 1,
    "name": "A green door",
    "price": 12.50,
    "tags": ["home", "green"]
    }
    '''

json_obj = ujson.loads(json_example)
json_obj

-------------------------------------------------


{'id': 1, 'name': 'A green door', 'price': 12.5, 'tags': ['home', 'green']}

## Broken tweets and irrelevant messages

The data of this assignment may contain broken tweets (invalid JSON strings). So make sure that your code is robust for such cases.

In addition, some lines in the input file might not be tweets, but messages that the Twitter server sent to the developer (such as [limit notices](https://dev.twitter.com/streaming/overview/messages-types#limit_notices)). Your program should also ignore these messages.

*Hint:* [Catch the ValueError](http://stackoverflow.com/questions/11294535/verify-if-a-string-is-json-in-python)


(1) Parse raw JSON tweets to obtain valid JSON objects. From all valid tweets, construct a pair RDD of `(user_id, text)`, where `user_id` is the `id_str` data field of the `user` dictionary (read [Tweets](#Tweets) section above), `text` is the `text` data field.


In [ ]:
import ujson

def safe_parse(raw_json):
    try:
        json_object = ujson.loads(raw_json)
    except ValueError:
        return []
    if 'user' not in json_object.keys():
        return []
    return [json_object]

# your code here

(2) Count the number of different users in all valid tweets (hint: [the `distinct()` method](https://spark.apache.org/docs/latest/programming-guide.html#transformations)).

It should print
```
The number of unique users is: 4
```

In [ ]:
def print_users_count(count):
    print ('The number of unique users is:', count)

# your code here

The number of unique users is: 4
